In [1]:
import mlflow

In [2]:
mlflow.set_tracking_uri("http://localhost:5000/")

In [3]:
mlflow.set_experiment("Loan experiment")

<Experiment: artifact_location='mlflow-artifacts:/957362813745767861', creation_time=1745664492937, experiment_id='957362813745767861', last_update_time=1745664492937, lifecycle_stage='active', name='Loan experiment', tags={}>

In [4]:
import pandas as pd
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from collections import Counter
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from sklearn.preprocessing import OrdinalEncoder

In [12]:
import re
data = pd.read_csv("Loan Prediction.csv")
data = data.drop("Id", axis=1)
print(data.head())
cat_cols = data.select_dtypes(include='object').columns.tolist()
pattern = re.compile(r"[^a-zA-Z]")
for col in cat_cols:
    data[col] = data[col].astype(str).apply(lambda x: pattern.sub("", x))
num_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()



    Income  Age  Experience Married/Single House_Ownership Car_Ownership  \
0  1303834   23           3         single          rented            no   
1  7574516   40          10         single          rented            no   
2  3991815   66           4        married          rented            no   
3  6256451   41           2         single          rented           yes   
4  5768871   47          11         single          rented            no   

            Profession                 CITY           STATE  CURRENT_JOB_YRS  \
0  Mechanical_engineer                 Rewa  Madhya_Pradesh                3   
1   Software_Developer             Parbhani     Maharashtra                9   
2     Technical_writer            Alappuzha          Kerala                4   
3   Software_Developer          Bhubaneswar          Odisha                2   
4        Civil_servant  Tiruchirappalli[10]      Tamil_Nadu                3   

   CURRENT_HOUSE_YRS  Risk_Flag  
0                 13        

In [10]:
ordinal_encoder = OrdinalEncoder()
X_trans = ordinal_encoder.fit_transform(data)
all_columns = num_cols + cat_cols
data = pd.DataFrame(X_trans, columns=all_columns)
data.head()

,Income,Age,Experience,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE
0,5343.0,2.0,3.0,1.0,2.0,0.0,33.0,250.0,13.0,3.0,3.0,0.0
1,31812.0,19.0,10.0,1.0,2.0,0.0,43.0,226.0,14.0,9.0,3.0,0.0
2,16782.0,45.0,4.0,0.0,2.0,0.0,47.0,8.0,12.0,4.0,0.0,0.0
3,26267.0,20.0,2.0,1.0,2.0,1.0,43.0,53.0,17.0,2.0,2.0,1.0
4,24217.0,26.0,11.0,1.0,2.0,0.0,11.0,295.0,22.0,3.0,4.0,1.0


In [14]:
X_trans.head()

,Income,Age,Experience,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE
0,5343.0,2.0,3.0,1.0,2.0,0.0,33.0,250.0,13.0,3.0,3.0
1,31812.0,19.0,10.0,1.0,2.0,0.0,43.0,226.0,14.0,9.0,3.0
2,16782.0,45.0,4.0,0.0,2.0,0.0,47.0,8.0,12.0,4.0,0.0
3,26267.0,20.0,2.0,1.0,2.0,1.0,43.0,53.0,17.0,2.0,2.0
4,24217.0,26.0,11.0,1.0,2.0,0.0,11.0,295.0,22.0,3.0,4.0


In [39]:
corr = X_trans.corr()
corr

,Income,Age,Experience,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE
Income,1.000000,-0.000696,0.006410,0.002706,0.016418,0.004051,0.000721,-0.001910,-0.004336,0.006974,-0.002388
Age,-0.000696,1.000000,-0.001118,-0.005323,-0.009820,0.009395,-0.008551,0.002101,-0.004979,0.002154,-0.020134
Experience,0.006410,-0.001118,1.000000,-0.001752,-0.007912,0.007519,0.000975,-0.025669,-0.000877,0.646098,0.019309
CURRENT_JOB_YRS,0.002706,-0.005323,-0.001752,1.000000,-0.022501,0.001206,-0.007534,0.012245,-0.008986,0.004251,-0.007539
CURRENT_HOUSE_YRS,0.016418,-0.009820,-0.007912,-0.022501,1.000000,-0.001233,0.001242,-0.014451,-0.007481,-0.009790,-0.005618
Married/Single,0.004051,0.009395,0.007519,0.001206,-0.001233,1.000000,0.011887,0.001630,0.026121,0.011099,0.000251
House_Ownership,0.000721,-0.008551,0.000975,-0.007534,0.001242,0.011887,1.000000,0.018845,0.001462,-0.004536,0.001954
Car_Ownership,-0.001910,0.002101,-0.025669,0.012245,-0.014451,0.001630,0.018845,1.000000,-0.036200,-0.027329,-0.010662
Profession,-0.004336,-0.004979,-0.000877,-0.008986,-0.007481,0.026121,0.001462,-0.036200,1.000000,0.009123,0.007572
CITY,0.006974,0.002154,0.646098,0.004251,-0.009790,0.011099,-0.004536,-0.027329,0.009123,1.000000,0.005372


In [40]:
abs(corr) > 0.7

,Income,Age,Experience,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE
Income,True,False,False,False,False,False,False,False,False,False,False
Age,False,True,False,False,False,False,False,False,False,False,False
Experience,False,False,True,False,False,False,False,False,False,False,False
CURRENT_JOB_YRS,False,False,False,True,False,False,False,False,False,False,False
CURRENT_HOUSE_YRS,False,False,False,False,True,False,False,False,False,False,False
Married/Single,False,False,False,False,False,True,False,False,False,False,False
House_Ownership,False,False,False,False,False,False,True,False,False,False,False
Car_Ownership,False,False,False,False,False,False,False,True,False,False,False
Profession,False,False,False,False,False,False,False,False,True,False,False
CITY,False,False,False,False,False,False,False,False,False,True,False


In [44]:
import matplotlib.pyplot as plt

In [59]:
risk_X = X_trans[y==1]
not_risk_X = X_trans[y==0]

In [63]:
half = pd.concat([risk_X, not_risk_X[:len(risk_X)]])
len(half)

61992

In [64]:
plt.scatter(half["Experience"].head(1000), falf["CITY"].head(1000), c=y.head(1000) )
plt.show()

NameError: name 'falf' is not defined